```pip install streamlit```<br>
```pip install ipykernel```<br>
```pip install pandas```<br>
```pip install geopy```

to run app :
```streamlit run app.py```

In [ ]:
import streamlit as st  # Import Streamlit for creating web apps
import datetime as dt  # Import datetime for working with dates and times
import pandas as pd  # Import pandas for data manipulation
import requests  # Import requests for making HTTP requests
import urllib  # Import module for working with URLs
import json  # Import module for working with JSON data

# import folium

In [57]:
station_url = 'https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_status'
latlon_url = 'https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information'

In [ ]:
def get_station_status(url):
    with urllib.request.urlopen(url) as data_url:  # Open the URL
        data = json.loads(data_url.read().decode())  # Read and decoded the JSON data
    
    df = pd.DataFrame(data['data']['stations'])  # Convert the json data into a dataframe
    df = df[df['is_renting'] == 1]  # Filter out the stations that are not renting
    df = df[df['is_returning'] == 1]  # Filter out the stations that are not returning
    df.drop_duplicates(['station_id', 'last_reported'])  # Remove duplicated records

    df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s', utc=True)  # Convert the timestamps in date and time
    df['current_time'] = data['last_updated']  # Store the API's last updated timestamp
    df['current_time'] = pd.to_datetime(df['last_reported'], unit='s', utc=True)  # Convert the timestamp to a timezone-aware UTC datetime

    df = df.set_index('current_time')  # Use the update time as the DataFrame index

    df = pd.concat([df, df['num_bikes_available_types'].apply(pd.Series)], axis=1)  # Expand the dict of bike types into separate columns and append them to df

    return df  # Return the DataFrame

get_station_status(station_url)


In [ ]:
def get_station_latlon(url):
    with urllib.request.urlopen(url) as latlon_url:  # Open the URL
        data = json.loads(latlon_url.read().decode())  # Read and decoded the JSON data

    df = pd.DataFrame(data['data']['stations'])  # Convert the json data into a dataframe
    ## Toronto's bikeshare guidelines say that an electric bike can be dropped at a non-charging station, so we do not need to know if it is a charging station
    
    return df

get_station_latlon(latlon_url)

In [59]:
def merge_df(df1, df2):
    df = df1.merge(df2, how='left', on='station_id')

    return df

In [ ]:
data_df = get_station_status(station_url)
latlon_df = get_station_latlon(latlon_url)
df = merge_df(data_df, latlon_df)
df = df.drop(columns= ['status', 'traffic', 'is_installed', 'is_renting', 'is_returning', 'last_reported', 'num_bikes_available_types', 'name', 'physical_configuration', 'groups', 'obcn', 'short_name', 'nearby_distance', 'address', 'is_charging_station', 'rental_methods', '_ride_code_support', 'rental_uris', 'post_code', 'altitude', 'is_valet_station', 'cross_street'])
df

,station_id,num_bikes_available,num_bikes_disabled,num_docks_available,num_docks_disabled,mechanical,ebike,lat,lon,capacity
0,7000,1,0,45,1,1,0,43.639832,-79.395954,47
1,7001,2,0,19,2,2,0,43.664964,-79.383550,23
2,7002,3,0,15,1,2,1,43.667131,-79.399555,19
3,7003,0,0,14,1,0,0,43.667018,-79.402796,15
4,7005,23,0,3,0,23,0,43.648001,-79.383177,26
...,...,...,...,...,...,...,...,...,...,...
1001,8190,8,0,17,2,8,0,43.666692,-79.314374,27
1002,8202,1,0,18,0,1,0,43.781009,-79.245315,19
1003,8203,0,0,11,0,0,0,43.760466,-79.229810,11
1004,8205,0,0,15,0,0,0,43.750777,-79.221929,15
